In [116]:
from os import listdir
from pickle import dump
import keras
import glob
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from tqdm import tqdm
import os
import pickle
import numpy as np

import os
import cv2
import numpy as np
import warnings
import time
import glob
from PIL import Image
from src.generate_patches import CropImage
import math
from tqdm import tqdm

In [70]:
class Detection:
    def __init__(self):
        caffemodel = "./resources/detection_model/Widerface-RetinaFace.caffemodel"
        deploy = "./resources/detection_model/deploy.prototxt"
        self.detector = cv2.dnn.readNetFromCaffe(deploy, caffemodel)
        self.detector_confidence = 0.9

    def get_bbox(self, img):
        height, width = img.shape[0], img.shape[1]
        aspect_ratio = width / height
        if img.shape[1] * img.shape[0] >= 192 * 192:
            img = cv2.resize(img,
                             (int(192 * math.sqrt(aspect_ratio)),
                              int(192 / math.sqrt(aspect_ratio))), interpolation=cv2.INTER_LINEAR)

        blob = cv2.dnn.blobFromImage(img, 1, mean=(104, 117, 123))
        self.detector.setInput(blob, 'data')
        out = self.detector.forward('detection_out').squeeze()
        max_conf_index = np.argmax(out[:, 2])
        left, top, right, bottom = out[max_conf_index, 3]*width, out[max_conf_index, 4]*height, \
                                   out[max_conf_index, 5]*width, out[max_conf_index, 6]*height
        bbox = [int(left), int(top), int(right-left+1), int(bottom-top+1)]
        return bbox

In [71]:
image_cropper = CropImage()
model_test = Detection()

In [72]:
def get_crop_face(frame):
    image_bbox = model_test.get_bbox(frame)
    h_input, w_input, _ = frame.shape
    param = {
    "org_img": frame,
    "bbox": image_bbox,
    "scale": 1.0,
    "out_w": w_input,
    "out_h": h_input,
    "crop": True,
    }
    crop = image_cropper.crop(**param)
    return crop, image_bbox

In [101]:
path='/home/mle/Desktop/fast.ai_course/face_extractor/extractor/dataset/test/spoof/anomaly/spoof.mp4'


In [102]:

cam = cv2.VideoCapture(path)


currentframe = 0

out_put_dir = f'dataset/test_spoof/'
os.makedirs(out_put_dir, exist_ok=True) 
val= []
face_crop = []
val = 0
#print("yes")
while(True): 

    # reading from frame 
    ret,img = cam.read() 

    if ret:
        frame = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        crop, image_bbox = get_crop_face(frame)
        if image_bbox == [0,0,1,1]:
            continue
        vidlast = path.split("/")[-1].split(".")[0]
        crop = cv2.resize(crop, (224,224))
        crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        cv2.imwrite(f"{out_put_dir}/{vidlast}_{val}.png",crop)
        currentframe += 1
        val = val+1
        #print(val)
    else: 
        break
    #break
cam.release() 
cv2.destroyAllWindows()

In [80]:
target_size=224
model = ResNet50()
#Modify model to remove the last layer
model.layers.pop()
model = Model(inputs=model.inputs,outputs=model.layers[-1].output)
# print(model.summary())

# extracting feature of real



In [167]:
test_img = "./dataset/test/real/aczrgyricp/"
# test_img = "./dataset/test_spoof/"

In [168]:
dirs = os.listdir(test_img)
for img in dirs:
    pass

In [169]:
features=[]
for img_name in tqdm(dirs):
    image = load_img(test_img+img_name,target_size=(target_size,target_size))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    img_feature = model.predict(image, verbose=0)
    # store feature
    features.append(img_feature)


100%|██████████| 115/115 [00:40<00:00,  2.96it/s]

In [170]:
filename = "best_model.pkl"
loaded_model = pickle.load(open(filename, 'rb'))

In [171]:
positive_features_test = [fes.reshape(-1) for fes in features]
predicted = loaded_model.predict(positive_features_test)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished


In [172]:
predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [173]:
result = np.average(predicted)

In [174]:
result

0.0

In [175]:
if result>0.5:
    print("Spoof")
else:
    print("Real")

Real
